# 02 - ML Experimentation with Custom Model

The purpose of this notebook is to use [custom training](https://cloud.google.com/ai-platform-unified/docs/training/custom-training) to train a keras classifier to predict whether a given trip will result in a tip > 20%. The notebook covers the following tasks:
1. Preprocess the data locally using Apache Beam.
2. Train and test custom model locally using a Keras implementation.
3. Submit a Dataflow job to preprocess the data at scale.
4. Submit a custom training job to Vertex AI using a [pre-built container](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).
5. Upload the trained model to Vertex AI.
6. Track experiment parameters from [Vertex AI Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction).
7. Submit a [hyperparameter tuning job](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) to Vertex AI.

We use [Vertex TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) 
and [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction) to  track, visualize, and compare ML experiments.

## Setup

### Import libraries

In [ ]:
pip install tensorrt

In [1]:
import os
import logging
from datetime import datetime
import numpy as np

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow.keras as keras

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import hyperparameter_tuning as hp_tuning

from src.common import features, datasource_utils
from src.model_training import data, model, defaults, trainer, exporter
from src.preprocessing import etl

logging.getLogger().setLevel(logging.INFO)
tf.get_logger().setLevel('INFO')

print(f"TensorFlow: {tf.__version__}")
print(f"TensorFlow Transform: {tft.__version__}")

2024-06-25 04:31:23.845231: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 04:31:25.396725: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow: 2.16.1
TensorFlow Transform: 1.15.0


### Setup Google Cloud project

In [2]:
PROJECT = 'supply-chain-twin-349311' # Change to your project id.
REGION = 'us-central1' # Change to your region.
BUCKET = 'test-lora' # Change to your bucket name.
SERVICE_ACCOUNT ="db-migration-genai@supply-chain-twin-349311.iam.gserviceaccount.com"

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP projet id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn'exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
PARENT = f"projects/{PROJECT}/locations/{REGION}"
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)
print("Vertex API Parent URI:", PARENT)

Project ID: supply-chain-twin-349311
Region: us-central1
Bucket name: test-lora
Service Account: db-migration-genai@supply-chain-twin-349311.iam.gserviceaccount.com
Vertex API Parent URI: projects/supply-chain-twin-349311/locations/us-central1


### Set configurations

In [3]:
VERSION = 'v09'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'

WORKSPACE = f'gs://{BUCKET}/{DATASET_DISPLAY_NAME}'
EXPERIMENT_ARTIFACTS_DIR = os.path.join(WORKSPACE, 'experiments')
RAW_SCHEMA_LOCATION = 'src/raw_schema/schema.pbtxt'

TENSORBOARD_DISPLAY_NAME = f'tb-{DATASET_DISPLAY_NAME}'
EXPERIMENT_NAME = f'{MODEL_DISPLAY_NAME}'

## Create Vertex TensorBoard instance 

In [4]:
tensorboard_resource = vertex_ai.Tensorboard.create(display_name=TENSORBOARD_DISPLAY_NAME)
tensorboard_resource_name = tensorboard_resource.gca_resource.name
print("TensorBoard resource name:", tensorboard_resource_name)

Creating Tensorboard
Create Tensorboard backing LRO: projects/1049330678395/locations/us-central1/tensorboards/5089019200417038336/operations/6528758517971550208
Tensorboard created. Resource name: projects/1049330678395/locations/us-central1/tensorboards/5089019200417038336
To use this Tensorboard in another session:
tb = aiplatform.Tensorboard('projects/1049330678395/locations/us-central1/tensorboards/5089019200417038336')
TensorBoard resource name: projects/1049330678395/locations/us-central1/tensorboards/5089019200417038336


## Initialize workspace

In [5]:
REMOVE_EXPERIMENT_ARTIFACTS = False

if tf.io.gfile.exists(EXPERIMENT_ARTIFACTS_DIR) and REMOVE_EXPERIMENT_ARTIFACTS:
    print("Removing previous experiment artifacts...")
    tf.io.gfile.rmtree(EXPERIMENT_ARTIFACTS_DIR)

if not tf.io.gfile.exists(EXPERIMENT_ARTIFACTS_DIR):
    print("Creating new experiment artifacts directory...")
    tf.io.gfile.mkdir(EXPERIMENT_ARTIFACTS_DIR)

print("Workspace is ready.")
print("Experiment directory:", EXPERIMENT_ARTIFACTS_DIR)

Creating new experiment artifacts directory...
Workspace is ready.
Experiment directory: gs://test-lora/chicago-taxi-tips/experiments


## Initialize Vertex AI experiment

In [6]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=BUCKET,
    experiment=EXPERIMENT_NAME
)

run_id = f"run-local-{datetime.now().strftime('%Y%m%d%H%M%S')}"
vertex_ai.start_run(run_id)

EXPERIMENT_RUN_DIR = os.path.join(EXPERIMENT_ARTIFACTS_DIR, EXPERIMENT_NAME, run_id)
print("Experiment run directory:", EXPERIMENT_RUN_DIR)

Associating projects/1049330678395/locations/us-central1/metadataStores/default/contexts/chicago-taxi-tips-classifier-v09-run-local-20240625043204 to Experiment: chicago-taxi-tips-classifier-v09


Experiment run directory: gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-local-20240625043204


## 1. Preprocess the data using Apache Beam

The Apache Beam pipeline of data preprocessing is implemented in the [preprocessing](src/preprocessing) directory.

In [7]:
EXPORTED_DATA_PREFIX = os.path.join(EXPERIMENT_RUN_DIR, 'exported_data')
TRANSFORMED_DATA_PREFIX = os.path.join(EXPERIMENT_RUN_DIR, 'transformed_data')
TRANSFORM_ARTIFACTS_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'transform_artifacts')

### Get Source Query from Managed Dataset

In [8]:
ML_USE = 'UNASSIGNED'
LIMIT = 5120

raw_data_query = datasource_utils.get_training_source_query(
    project=PROJECT, 
    region=REGION, 
    dataset_display_name=DATASET_DISPLAY_NAME, 
    ml_use=ML_USE, 
    limit=LIMIT
)

print(raw_data_query)


    SELECT 
        IF(trip_month IS NULL, -1, trip_month) trip_month,
        IF(trip_day IS NULL, -1, trip_day) trip_day,
        IF(trip_day_of_week IS NULL, -1, trip_day_of_week) trip_day_of_week,
        IF(trip_hour IS NULL, -1, trip_hour) trip_hour,
        IF(trip_seconds IS NULL, -1, trip_seconds) trip_seconds,
        IF(trip_miles IS NULL, -1, trip_miles) trip_miles,
        IF(payment_type IS NULL, 'NA', payment_type) payment_type,
        IF(pickup_grid IS NULL, 'NA', pickup_grid) pickup_grid,
        IF(dropoff_grid IS NULL, 'NA', dropoff_grid) dropoff_grid,
        IF(euclidean IS NULL, -1, euclidean) euclidean,
        IF(loc_cross IS NULL, 'NA', loc_cross) loc_cross,
        tip_bin
    FROM chi_e2e.chicago_taxitrips_prep 
    WHERE ML_use = 'UNASSIGNED'
    LIMIT 5120


### Test Data Preprocessing Locally

In [9]:
args = {
    'runner': 'DirectRunner',
    'raw_data_query': raw_data_query,
    'write_raw_data': True,
    'exported_data_prefix': EXPORTED_DATA_PREFIX,
    'transformed_data_prefix': TRANSFORMED_DATA_PREFIX,
    'transform_artifact_dir': TRANSFORM_ARTIFACTS_DIR,
    'temporary_dir': os.path.join(WORKSPACE, 'tmp'),
    'gcs_location': f'gs://{BUCKET}/bq_tmp',
    'project': PROJECT
}

In [10]:
vertex_ai.log_params(args)

In [11]:
print("Data preprocessing started...")
etl.run_transform_pipeline(args)
print("Data preprocessing completed.")

INFO:absl:Feature trip_month has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_day has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_day_of_week has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_hour has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_seconds has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_miles has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature payment_type has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature pickup_grid has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature dropoff_grid has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature euclidean has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature loc_cross has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature tip_bin has a shape dim {
  size: 1
}
.

Data preprocessing started...


INFO:absl:Feature dropoff_grid has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature euclidean has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature loc_cross has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature payment_type has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature pickup_grid has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature tip_bin has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_day has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_day_of_week has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_hour has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_miles has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_month has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_seconds has a shape dim {
  size: 1
}
.

INFO:tensorflow:Assets written to: gs://test-lora/chicago-taxi-tips/tmp/tftransform_tmp/2f7d67b7cb3b40c39935a6a535938e04/assets


INFO:tensorflow:Assets written to: gs://test-lora/chicago-taxi-tips/tmp/tftransform_tmp/2f7d67b7cb3b40c39935a6a535938e04/assets


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:apache_beam.io.gcp.gcsio:Finished listing 1 files in 0.03151345252990723 seconds.
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.32 seconds.
INFO:apache_beam.io.gcp.gcsio:Finished listing 1 files in 0.03499770164489746 seconds.
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.32 seconds.


Data preprocessing completed.


In [12]:
!gsutil ls {EXPERIMENT_RUN_DIR}

gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-local-20240625043204/exported_data/
gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-local-20240625043204/transform_artifacts/
gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-local-20240625043204/transformed_data/


## 2. Train a custom model locally using a Keras

The `Keras` implementation of the custom model is in the [model_training](src/model_training) directory.

In [13]:
LOG_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'logs')
EXPORT_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'model')

### Read transformed data

In [14]:
tft_output = tft.TFTransformOutput(TRANSFORM_ARTIFACTS_DIR)
print(tft_output)
transform_feature_spec = tft_output.transformed_feature_spec()
transform_feature_spec

{'dropoff_grid_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'euclidean_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'loc_cross_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'payment_type_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'pickup_grid_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'tip_bin': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'trip_day_of_week_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'trip_day_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'trip_hour_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'trip_miles_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'trip_month_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'trip_seconds_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None)}

In [15]:
train_data_file_pattern = os.path.join(TRANSFORMED_DATA_PREFIX,'train/data-*.gz')
eval_data_file_pattern = os.path.join(TRANSFORMED_DATA_PREFIX,'eval/data-*.gz')

for input_features, target in data.get_dataset(
    train_data_file_pattern, transform_feature_spec, batch_size=3).take(1):
    for key in input_features:
        print(f"{key} {input_features[key].dtype}: {input_features[key].numpy().tolist()}")
    print(f"target: {target.numpy().tolist()}")

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


dropoff_grid_xf <dtype: 'int64'>: [0, 0, 0]
euclidean_xf <dtype: 'float32'>: [1.9481756687164307, -0.8376971483230591, -0.8376971483230591]
loc_cross_xf <dtype: 'int64'>: [0, 0, 0]
payment_type_xf <dtype: 'int64'>: [0, 0, 0]
pickup_grid_xf <dtype: 'int64'>: [0, 0, 0]
trip_day_of_week_xf <dtype: 'int64'>: [3, 4, 5]
trip_day_xf <dtype: 'int64'>: [20, 11, 7]
trip_hour_xf <dtype: 'int64'>: [15, 2, 0]
trip_miles_xf <dtype: 'float32'>: [1.7764558792114258, 1.7454386949539185, -0.6694629192352295]
trip_month_xf <dtype: 'int64'>: [4, 3, 2]
trip_seconds_xf <dtype: 'float32'>: [0.26116666197776794, 0.5803168416023254, -0.1925041377544403]
target: [0, 0, 0]


2024-06-25 04:35:38.796923: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Create hyperparameters

In [16]:
hyperparams = {
    "hidden_units": [64, 32]
}

hyperparams = defaults.update_hyperparams(hyperparams)
hyperparams

{'hidden_units': [64, 32],
 'learning_rate': 0.0001,
 'batch_size': 512,
 'num_epochs': 10}

In [ ]:
import tensorflow as tf
from tensorflow import keras

from src.common import features

def create_model_inputs():
    inputs = {}
    for feature_name in features.FEATURE_NAMES:
        name = features.transformed_name(feature_name)
        if feature_name in features.NUMERICAL_FEATURE_NAMES:
            inputs[name] = keras.layers.Input(name=name, shape=[], dtype=tf.float32)
        elif feature_name in features.categorical_feature_names():
            inputs[name] = keras.layers.Input(name=name, shape=[], dtype=tf.int64)
        else:
            pass
    return inputs

def _create_binary_classifier(feature_vocab_sizes, hyperparams):
    input_layers = create_model_inputs()

    layers = []
    for key in input_layers:
        feature_name = features.original_name(key)
        if feature_name in features.EMBEDDING_CATEGORICAL_FEATURES:
            vocab_size = feature_vocab_sizes[feature_name]
            embedding_size = features.EMBEDDING_CATEGORICAL_FEATURES[feature_name]
            embedding_output = keras.layers.Embedding(
                input_dim=vocab_size + 1,
                output_dim=embedding_size,
                name=f"{key}_embedding",
            )(input_layers[key])
            layers.append(embedding_output)
        elif feature_name in features.ONEHOT_CATEGORICAL_FEATURE_NAMES:
            vocab_size = feature_vocab_sizes[feature_name]
            onehot_layer = keras.layers.CategoryEncoding(
                num_tokens=vocab_size,
                output_mode="one_hot",
                name=f"{key}_onehot",
            )(input_layers[key])
            layers.append(onehot_layer)
        elif feature_name in features.NUMERICAL_FEATURE_NAMES:
            numeric_layer = tf.expand_dims(input_layers[key], -1)
            layers.append(numeric_layer)
        else:
            pass

    joined = keras.layers.Concatenate(name="combines_inputs")(layers)
    feedforward_output = keras.Sequential(
        [
            keras.layers.Dense(units, activation="relu")
            for units in hyperparams["hidden_units"]
        ],
        name="feedforward_network",
    )(joined)
    logits = keras.layers.Dense(units=1, name="logits")(feedforward_output)

    model = keras.Model(inputs=input_layers, outputs=[logits])
    return model

def create_binary_classifier(tft_output, hyperparams):
    feature_vocab_sizes = dict()
    for feature_name in features.categorical_feature_names():
        feature_vocab_sizes[feature_name] = tft_output.vocabulary_size_by_name(
            feature_name
        )

    return _create_binary_classifier(feature_vocab_sizes, hyperparams)
classifier = create_binary_classifier(tft_output, hyperparams)
classifier.summary()

### Create and test model inputs and outputs

In [17]:
classifier = model.create_binary_classifier(tft_output, hyperparams)
classifier.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ trip_month_xf       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trip_day_xf         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trip_day_of_week_xf │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trip_hour_xf        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ payment_type_xf     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pickup_grid_xf      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropoff_grid_xf     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ loc_cross_xf        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trip_month_xf_embe… │ (None, 1, 4)      │         44 │ trip_month_xf[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trip_day_xf_embedd… │ (None, 1, 30)     │        960 │ trip_day_xf[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trip_day_of_week_x… │ (None, 7)         │          0 │ trip_day_of_week… │
│ (CategoryEncoding)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trip_hour_xf_embed… │ (None, 1, 9)      │        225 │ trip_hour_xf[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trip_seconds_xf     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trip_miles_xf       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ payment_type_xf_on… │ (None, 7)         │          0 │ payment_type_xf[… │
│ (CategoryEncoding)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pickup_grid_xf_emb… │ (None, 1, 12)     │         36 │ pickup_grid_xf[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropoff_grid_xf_em… │ (None, 1, 12)     │         60 │ dropoff_grid_xf[

 Total params: 14,842 (57.98 KB)

 Trainable params: 14,842 (57.98 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
keras.utils.plot_model(
    classifier, 
    show_shapes=True, 
    show_dtype=True
)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [19]:
classifier(input_features)

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[ 0.18513012],
       [ 0.14987227],
       [-0.08429831]], dtype=float32)>

### Train the model locally.

In [20]:
logging.getLogger().setLevel(logging.INFO)
hyperparams = {
    "learning_rate": 0.001,
    "num_epochs": 5,
    "batch_size": 512,
    "hidden_units": [64, 32],
}

# Convert 'hidden_units' list to string for logging
hyperparams_for_logging = hyperparams.copy()
hyperparams_for_logging["hidden_units"] = str(hyperparams["hidden_units"])

# Log the hyperparameters
vertex_ai.log_params(hyperparams_for_logging)

In [21]:
classifier = trainer.train(
    train_data_dir=train_data_file_pattern,
    eval_data_dir=eval_data_file_pattern,
    tft_output_dir=TRANSFORM_ARTIFACTS_DIR,
    hyperparams=hyperparams,
    log_dir=LOG_DIR,
)

INFO:root:Loading tft output from gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-local-20240625043204/transform_artifacts
INFO:root:Model training started...


Epoch 1/5
      1/Unknown 3s 3s/step - accuracy: 0.8848 - loss: 0.6693

2024-06-25 04:36:27.464033: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-06-25 04:36:28.466518: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 264ms/step - accuracy: 0.8685 - loss: 0.5973 - val_accuracy: 0.4385 - val_loss: 0.2068
Epoch 2/5
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.8945 - loss: 0.3887

2024-06-25 04:36:29.431979: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-06-25 04:36:29.570211: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


...

2024-06-25 04:36:31.624808: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-06-25 04:36:31.733985: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.8732 - loss: 0.2713 - val_accuracy: 0.4404 - val_loss: 0.1338
Epoch 5/5
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.9102 - loss: 0.2190

2024-06-25 04:36:32.661801: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-06-25 04:36:32.764629: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.8753 - loss: 0.2403 - val_accuracy: 0.4385 - val_loss: 0.1228


INFO:root:Model training completed.


In [22]:
val_loss, val_accuracy = trainer.evaluate(
    model=classifier,
    data_dir=eval_data_file_pattern,
    raw_schema_location=RAW_SCHEMA_LOCATION,
    tft_output_dir=TRANSFORM_ARTIFACTS_DIR,
    hyperparams=hyperparams,
)

INFO:root:Loading raw schema from src/raw_schema/schema.pbtxt
INFO:root:Loading tft output from gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-local-20240625043204/transform_artifacts
INFO:root:Model evaluation started...


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.6519 - loss: 0.1945


2024-06-25 04:36:37.396192: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
INFO:root:Model evaluation completed.


In [23]:
vertex_ai.log_metrics(
    {"val_loss": val_loss, "val_accuracy": val_accuracy})

In [ ]:
pip install tensorboard-plugin-profile

In [24]:
!tb-gcp-uploader --tensorboard_resource_name={tensorboard_resource_name} \
  --logdir={LOG_DIR} \
  --experiment_name={EXPERIMENT_NAME} --one_shot=True

2024-06-25 04:36:49.492899: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 04:36:51.005156: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
<IPython.core.display.HTML object>
View your Tensorboard at https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1049330678395+locations+us-central1+tensorboards+5089019200417038336+experiments+chicago-taxi-tips-classifier-v09
W0625 04:36:52.204046 140387284313920 uploader.py:327] Please consider uploading to a new experiment instead of an existing one, as the former allows for better upload performance.
[2024-06-25T04:36:52] Started scanning logdir.
[2024-06-25T04:36:55] Total uploaded: 35 scalars, 0 tensors, 1 binary objects (165.6 kB)

In [ ]:
import logging
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_data_validation as tfdv
from tensorflow_transform.tf_metadata import schema_utils
import tensorflow.keras as keras

from src.common import features

def _get_serve_tf_examples_fn(classifier, tft_output, raw_feature_spec):
    classifier.tft_layer = tft_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        for key in list(raw_feature_spec.keys()):
            if key not in features.FEATURE_NAMES:
                raw_feature_spec.pop(key)

        parsed_features = tf.io.parse_example(serialized_tf_examples, raw_feature_spec)
        transformed_features = classifier.tft_layer(parsed_features)
        logits = classifier(transformed_features)
        probabilities = keras.activations.sigmoid(logits)
        return {"probabilities": probabilities}

    return serve_tf_examples_fn

def _get_serve_features_fn(classifier, tft_output):
    classifier.tft_layer = tft_output.transform_features_layer()

    @tf.function
    def serve_features_fn(raw_features):
        transformed_features = classifier.tft_layer(raw_features)
        logits = classifier(transformed_features)
        neg_probabilities = keras.activations.sigmoid(logits)
        pos_probabilities = 1 - neg_probabilities
        probabilities = tf.concat([neg_probabilities, pos_probabilities], -1)
        batch_size = tf.shape(probabilities)[0]
        classes = tf.repeat([features.TARGET_LABELS], [batch_size], axis=0)
        return {"classes": classes, "scores": probabilities}

    return serve_features_fn

def export_serving_model(classifier, serving_model_dir, raw_schema_location, tft_output_dir):
    raw_schema = tfdv.load_schema_text(raw_schema_location)
    raw_feature_spec = schema_utils.schema_as_feature_spec(raw_schema).feature_spec

    tft_output = tft.TFTransformOutput(tft_output_dir)

    features_input_signature = {
        feature_name: tf.TensorSpec(
            shape=(None, 1), dtype=spec.dtype, name=feature_name
        )
        for feature_name, spec in raw_feature_spec.items()
        if feature_name in features.FEATURE_NAMES
    }

    serve_features_fn = _get_serve_features_fn(classifier, tft_output)
    serve_tf_examples_fn = _get_serve_tf_examples_fn(classifier, tft_output, raw_feature_spec)

    logging.info("Model export started...")

    # Remove the existing 'signatures' attribute if it exists
    if hasattr(classifier, 'signatures'):
        delattr(classifier, 'signatures')

    # Save the model for serving
    tf.saved_model.save(
        classifier,
        serving_model_dir,
        signatures={
            "serving_default": serve_features_fn.get_concrete_function(features_input_signature),
            "serving_tf_example": serve_tf_examples_fn.get_concrete_function(
                tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
            ),
        }
    )
    
    logging.info("Model export completed.")



saved_model_dir = os.path.join(EXPORT_DIR)
export_serving_model(
    classifier=classifier,
    serving_model_dir=saved_model_dir,
    raw_schema_location=RAW_SCHEMA_LOCATION,
    tft_output_dir=TRANSFORM_ARTIFACTS_DIR,
)

### Export the trained model

In [25]:
saved_model_dir = os.path.join(EXPORT_DIR)

exporter.export_serving_model(
    classifier=classifier,
    serving_model_dir=saved_model_dir,
    raw_schema_location=RAW_SCHEMA_LOCATION,
    tft_output_dir=TRANSFORM_ARTIFACTS_DIR,
)

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:root:Model export started...


INFO:tensorflow:Assets written to: gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-local-20240625043204/model/assets


INFO:tensorflow:Assets written to: gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-local-20240625043204/model/assets
INFO:root:Model export completed.


### Inspect model serving signatures

In [26]:
!saved_model_cli show --dir={saved_model_dir} --tag_set=serve --signature_def=serving_tf_example

2024-06-25 04:38:03.062391: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 04:38:04.304742: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The given SavedModel SignatureDef contains the following input(s):
  inputs['examples'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_tf_example_examples:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['probabilities'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall_25:0
Method name is: tensorflow/serving/predict


In [27]:
!saved_model_cli show --dir={saved_model_dir} --tag_set=serve --signature_def=serving_default

2024-06-25 04:38:14.940217: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 04:38:16.168220: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The given SavedModel SignatureDef contains the following input(s):
  inputs['dropoff_grid'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_dropoff_grid:0
  inputs['euclidean'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_euclidean:0
  inputs['loc_cross'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_loc_cross:0
  inputs['payment_type'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_payment_type:0
  inputs['

### Test the exported SavedModel

In [28]:
serving_model = tf.saved_model.load(saved_model_dir)
print("Saved model is loaded.")

Saved model is loaded.


In [29]:
# Test the serving_tf_example with TF Examples

file_names = tf.data.TFRecordDataset.list_files(EXPORTED_DATA_PREFIX + '/data-*.tfrecord')
for batch in tf.data.TFRecordDataset(file_names).batch(3).take(1):
    predictions = serving_model.signatures['serving_tf_example'](batch)
    for key in predictions:
        print(f"{key}: {predictions[key]}")

probabilities: [[0.04186616]
 [0.24097115]
 [0.25058404]]


2024-06-25 04:38:34.558507: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [30]:
# Test the serving_default with feature dictionary

import tensorflow_data_validation as tfdv
from tensorflow_transform.tf_metadata import schema_utils

raw_schema = tfdv.load_schema_text(RAW_SCHEMA_LOCATION)
raw_feature_spec = schema_utils.schema_as_feature_spec(raw_schema).feature_spec

In [31]:
instance = {
    "dropoff_grid": "POINT(-87.6 41.9)",
    "euclidean": 2064.2696,
    "loc_cross": "",
    "payment_type": "Credit Card",
    "pickup_grid": "POINT(-87.6 41.9)",
    "trip_miles": 1.37,
    "trip_day": 12,
    "trip_hour": 6,
    "trip_month": 2,
    "trip_day_of_week": 4,
    "trip_seconds": 555,
}

for feature_name in instance:
    dtype = raw_feature_spec[feature_name].dtype
    instance[feature_name] = tf.constant([[instance[feature_name]]], dtype)

In [32]:
predictions = serving_model.signatures['serving_default'](**instance)
for key in predictions:
    print(f"{key}: {predictions[key].numpy()}")

scores: [[0.5094008  0.49059922]]
classes: [[b'tip<20%' b'tip>=20%']]


## Start a new Vertex AI experiment run

In [33]:
vertex_ai.init(
    project=PROJECT,
    staging_bucket=BUCKET,
    experiment=EXPERIMENT_NAME)

run_id = f"run-gcp-{datetime.now().strftime('%Y%m%d%H%M%S')}"
vertex_ai.start_run(run_id)

EXPERIMENT_RUN_DIR = os.path.join(EXPERIMENT_ARTIFACTS_DIR, EXPERIMENT_NAME, run_id)
print("Experiment run directory:", EXPERIMENT_RUN_DIR)

Associating projects/1049330678395/locations/us-central1/metadataStores/default/contexts/chicago-taxi-tips-classifier-v09-run-gcp-20240625043856 to Experiment: chicago-taxi-tips-classifier-v09


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/1049330678395/locations/us-central1/metadataStores/default/contexts/chicago-taxi-tips-classifier-v09-run-gcp-20240625043856 to Experiment: chicago-taxi-tips-classifier-v09


Experiment run directory: gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856


## 3. Submit a Data Processing Job to Dataflow

In [34]:
EXPORTED_DATA_PREFIX = os.path.join(EXPERIMENT_RUN_DIR, 'exported_data')
TRANSFORMED_DATA_PREFIX = os.path.join(EXPERIMENT_RUN_DIR, 'transformed_data')
TRANSFORM_ARTIFACTS_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'transform_artifacts')

In [35]:
ML_USE = 'UNASSIGNED'
LIMIT = 1000000
raw_data_query = datasource_utils.get_training_source_query(
    project=PROJECT, 
    region=REGION, 
    dataset_display_name=DATASET_DISPLAY_NAME, 
    ml_use=ML_USE, 
    limit=LIMIT
)

etl_job_name = f"etl-{MODEL_DISPLAY_NAME}-{run_id}"

args = {
    'job_name': etl_job_name,
    'runner': 'DataflowRunner',
    'raw_data_query': raw_data_query,
    'exported_data_prefix': EXPORTED_DATA_PREFIX,
    'transformed_data_prefix': TRANSFORMED_DATA_PREFIX,
    'transform_artifact_dir': TRANSFORM_ARTIFACTS_DIR,
    'write_raw_data': False,
    'temporary_dir': os.path.join(WORKSPACE, 'tmp'),
    'gcs_location': os.path.join(WORKSPACE, 'bq_tmp'),
    'project': PROJECT,
    'region': REGION,
    'setup_file': './setup.py'
}

In [36]:
vertex_ai.log_params(args)

In [ ]:
logging.getLogger().setLevel(logging.ERROR)

print("Data preprocessing started...")
etl.run_transform_pipeline(args)
print("Data preprocessing completed.")

Data preprocessing started...


/opt/conda/bin/python: No module named build.__main__; 'build' is a package and cannot be directly executed


Data preprocessing completed.


In [38]:
!gsutil ls {EXPERIMENT_RUN_DIR}

gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856/
gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856/transform_artifacts/
gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856/transformed_data/


## 4. Submit a Custom Training Job to Vertex AI

In [39]:
LOG_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'logs')
EXPORT_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'model')

### Test the training task locally

In [ ]:
pip install hypertune

In [40]:
!python -m src.model_training.task \
    --model-dir={EXPORT_DIR} \
    --log-dir={LOG_DIR} \
    --train-data-dir={TRANSFORMED_DATA_PREFIX}/train/* \
    --eval-data-dir={TRANSFORMED_DATA_PREFIX}/eval/*  \
    --tft-output-dir={TRANSFORM_ARTIFACTS_DIR} \
    --num-epochs=3 \
    --hidden-units=32,32 \
    --experiment-name={EXPERIMENT_NAME} \
    --run-name={run_id} \
    --project={PROJECT} \
    --region={REGION} \
    --staging-bucket={BUCKET}

2024-06-25 04:59:27.012800: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 04:59:28.366204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:root:Python Version = 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
INFO:root:TensorFlow Version = 2.16.1
INFO:root:TF_CONFIG = Not found
INFO:root:DEVICES = [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6834850756421695930
xla_global_id: -1
]
INFO:root:Task started...
INFO:root:Hyperparameter: {'model_dir': 'gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856/model', 'log_dir': 'gs://test-lora/chicago-taxi-tips/experiments/chi

### Prepare training package

In [41]:
TRAINER_PACKAGE_DIR = os.path.join(WORKSPACE, 'trainer_packages')
TRAINER_PACKAGE_NAME = f'{MODEL_DISPLAY_NAME}_trainer'
print("Trainer package upload location:", TRAINER_PACKAGE_DIR)

Trainer package upload location: gs://test-lora/chicago-taxi-tips/trainer_packages


In [42]:
!rm -r src/__pycache__/
!rm -r src/.ipynb_checkpoints/
!rm -r src/raw_schema/.ipynb_checkpoints/
!rm -f {TRAINER_PACKAGE_NAME}.tar {TRAINER_PACKAGE_NAME}.tar.gz

!mkdir {TRAINER_PACKAGE_NAME}

!cp setup.py {TRAINER_PACKAGE_NAME}/
!cp -r src {TRAINER_PACKAGE_NAME}/
!tar cvf {TRAINER_PACKAGE_NAME}.tar {TRAINER_PACKAGE_NAME}
!gzip {TRAINER_PACKAGE_NAME}.tar
!gsutil cp {TRAINER_PACKAGE_NAME}.tar.gz {TRAINER_PACKAGE_DIR}/
!rm -r {TRAINER_PACKAGE_NAME}
!rm -r {TRAINER_PACKAGE_NAME}.tar.gz

rm: cannot remove 'src/.ipynb_checkpoints/': No such file or directory
rm: cannot remove 'src/raw_schema/.ipynb_checkpoints/': No such file or directory
chicago-taxi-tips-classifier-v09_trainer/
chicago-taxi-tips-classifier-v09_trainer/setup.py
chicago-taxi-tips-classifier-v09_trainer/src/
chicago-taxi-tips-classifier-v09_trainer/src/common/
chicago-taxi-tips-classifier-v09_trainer/src/common/datasource_utils.py
chicago-taxi-tips-classifier-v09_trainer/src/common/.ipynb_checkpoints/
chicago-taxi-tips-classifier-v09_trainer/src/common/.ipynb_checkpoints/features-checkpoint.py
chicago-taxi-tips-classifier-v09_trainer/src/common/.ipynb_checkpoints/datasource_utils-checkpoint.py
chicago-taxi-tips-classifier-v09_trainer/src/common/features.py
chicago-taxi-tips-classifier-v09_trainer/src/common/__init__.py
chicago-taxi-tips-classifier-v09_trainer/src/common/__pycache__/
chicago-taxi-tips-classifier-v09_trainer/src/common/__pycache__/datasource_utils.cpython-310.pyc
chicago-taxi-tips-classifi

### Prepare the training job

In [59]:
TRAIN_RUNTIME = 'tf-cpu.2-7'
TRAIN_IMAGE = f"us-docker.pkg.dev/vertex-ai/training/{TRAIN_RUNTIME}:latest"
print("Training image:", TRAIN_IMAGE)

Training image: us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest


In [60]:
num_epochs = 10
learning_rate = 0.001
hidden_units = "64,64"

trainer_args = [
    f'--train-data-dir={TRANSFORMED_DATA_PREFIX + "/train/*"}',
    f'--eval-data-dir={TRANSFORMED_DATA_PREFIX + "/eval/*"}',
    f'--tft-output-dir={TRANSFORM_ARTIFACTS_DIR}',
    f'--num-epochs={num_epochs}',
    f'--learning-rate={learning_rate}',
    f'--project={PROJECT}',
    f'--region={REGION}',
    f'--staging-bucket={BUCKET}',
    f'--experiment-name={EXPERIMENT_NAME}'
]

In [61]:
package_uri = os.path.join(TRAINER_PACKAGE_DIR, f'{TRAINER_PACKAGE_NAME}.tar.gz')

worker_pool_specs = [
    {
        "replica_count": 1,
        "machine_spec": {
            "machine_type": 'n1-standard-16',
            "accelerator_count": 0
    },
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [package_uri],
            "python_module": "src.model_training.task",
            "args": trainer_args,
        }
    }
]

### Submit the training job

In [ ]:
print("Submitting a custom training job...")

training_job_display_name = f"{TRAINER_PACKAGE_NAME}_{run_id}"

training_job = vertex_ai.CustomJob(
    display_name=training_job_display_name,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=EXPERIMENT_RUN_DIR,
)

training_job.run(
    service_account=SERVICE_ACCOUNT,
    tensorboard=tensorboard_resource_name,
    sync=True
)

Submitting a custom training job...
Creating CustomJob


INFO:google.cloud.aiplatform.jobs:Creating CustomJob


CustomJob created. Resource name: projects/1049330678395/locations/us-central1/customJobs/276843593216294912


INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/1049330678395/locations/us-central1/customJobs/276843593216294912


To use this CustomJob in another session:


...

INFO:google.cloud.aiplatform.jobs:CustomJob projects/1049330678395/locations/us-central1/customJobs/276843593216294912 current state:
JobState.JOB_STATE_RUNNING


CustomJob projects/1049330678395/locations/us-central1/customJobs/276843593216294912 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:CustomJob projects/1049330678395/locations/us-central1/customJobs/276843593216294912 current state:
JobState.JOB_STATE_RUNNING


CustomJob projects/1049330678395/locations/us-central1/customJobs/276843593216294912 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:CustomJob projects/1049330678395/locations/us-central1/customJobs/276843593216294912 current state:
JobState.JOB_STATE_RUNNING


## 5. Upload exported model to Vertex AI Models

In [63]:
!gsutil ls {EXPORT_DIR}

gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856/model/
gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856/model/fingerprint.pb
gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856/model/saved_model.pb
gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856/model/assets/
gs://test-lora/chicago-taxi-tips/experiments/chicago-taxi-tips-classifier-v09/run-gcp-20240625043856/model/variables/


### Generate the Explanation metadata

In [64]:
explanation_config = features.generate_explanation_config()
explanation_config

{'inputs': {'trip_month': {'input_tensor_name': 'trip_month',
   'encoding': 'IDENTITY',
   'modality': 'categorical'},
  'trip_day': {'input_tensor_name': 'trip_day',
   'encoding': 'IDENTITY',
   'modality': 'categorical'},
  'trip_day_of_week': {'input_tensor_name': 'trip_day_of_week',
   'encoding': 'IDENTITY',
   'modality': 'categorical'},
  'trip_hour': {'input_tensor_name': 'trip_hour',
   'encoding': 'IDENTITY',
   'modality': 'categorical'},
  'trip_seconds': {'input_tensor_name': 'trip_seconds', 'modality': 'numeric'},
  'trip_miles': {'input_tensor_name': 'trip_miles', 'modality': 'numeric'},
  'payment_type': {'input_tensor_name': 'payment_type',
   'encoding': 'IDENTITY',
   'modality': 'categorical'},
  'pickup_grid': {'input_tensor_name': 'pickup_grid',
   'encoding': 'IDENTITY',
   'modality': 'categorical'},
  'dropoff_grid': {'input_tensor_name': 'dropoff_grid',
   'encoding': 'IDENTITY',
   'modality': 'categorical'},
  'euclidean': {'input_tensor_name': 'euclidean'

### Upload model

In [65]:
SERVING_RUNTIME='tf2-cpu.2-7'
SERVING_IMAGE = f"us-docker.pkg.dev/vertex-ai/prediction/{SERVING_RUNTIME}:latest"
print("Serving image:", SERVING_IMAGE)

Serving image: us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest


In [66]:
explanation_metadata = vertex_ai.explain.ExplanationMetadata(
    inputs=explanation_config["inputs"],
    outputs=explanation_config["outputs"],
)
explanation_parameters = vertex_ai.explain.ExplanationParameters(
    explanation_config["params"]
)

vertex_model = vertex_ai.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=EXPORT_DIR,
    serving_container_image_uri=SERVING_IMAGE,
    parameters_schema_uri=None,
    instance_schema_uri=None,
    explanation_metadata=explanation_metadata,
    explanation_parameters=explanation_parameters,
    labels={
        'dataset_name': DATASET_DISPLAY_NAME,
        'experiment': run_id
    }
)

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/1049330678395/locations/us-central1/models/7437212098997780480/operations/6311565389067190272


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/1049330678395/locations/us-central1/models/7437212098997780480/operations/6311565389067190272


Model created. Resource name: projects/1049330678395/locations/us-central1/models/7437212098997780480@1


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/1049330678395/locations/us-central1/models/7437212098997780480@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/1049330678395/locations/us-central1/models/7437212098997780480@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/1049330678395/locations/us-central1/models/7437212098997780480@1')


In [67]:
vertex_model.gca_resource

name: "projects/1049330678395/locations/us-central1/models/7437212098997780480@1"
display_name: "chicago-taxi-tips-classifier-v09"
predict_schemata {
}
metadata {
}
container_spec {
  image_uri: "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest"
}
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_deployment_resources_types: SHARED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
supported_output_storage_formats: "bigquery"
create_time {
  seconds: 1719305789
  nanos: 690003000
}
update_time {
  seconds: 1719306020
  nanos: 504500000
}
etag: "AMEw9yN015dTZf6BOOYjKDbzFjXQvC6rTxbDdlrjSp4gKCqSHivwAyf_R5Su2-gb46gF"
labels {
  key: "dataset_name"
  value: "chicago-taxi-tips"
}
labels {
  key: "experimen

## 6. Extract experiment run parameters

In [68]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

,0,1,2,3,4,5,6,7
experiment_name,chicago-taxi-tips-classifier-v09,chicago-taxi-tips-classifier-v09,chicago-taxi-tips-classifier-v09,chicago-taxi-tips-classifier-v09,chicago-taxi-tips-classifier-v09,chicago-taxi-tips-classifier-v09,chicago-taxi-tips-classifier-v09,chicago-taxi-tips-classifier-v09
run_name,run-gcp-20240625063350,run-gcp-20240625043856,run-local-20240625043204,run-gcp-20240625030906,validation,train,run-local-20240625025958,run-gcp-20240625080524
run_type,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.Run
state,RUNNING,RUNNING,RUNNING,RUNNING,COMPLETE,COMPLETE,RUNNING,STATE_UNSPECIFIED
param.temporary_dir,NaN,gs://test-lora/chicago-taxi-tips/tmp,gs://test-lora/chicago-taxi-tips/tmp,NaN,NaN,NaN,gs://test-lora/chicago-taxi-tips/tmp,NaN
param.region,NaN,us-central1,NaN,NaN,NaN,NaN,NaN,us-central1
param.project,NaN,supply-chain-twin-349311,supply-chain-twin-349311,NaN,NaN,NaN,supply-chain-twin-349311,supply-chain-twin-349311
param.write_raw_data,NaN,False,True,NaN,NaN,NaN,True,NaN
param.runner,NaN,DataflowRunner,DirectRunner,NaN,NaN,NaN,DirectRunner,NaN
param.job_name,NaN,etl-chicago-taxi-tips-classifier-v09-run-gcp-2...,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
print("Vertex AI Experiments:")
print(
    f"https://console.cloud.google.com/vertex-ai/locations{REGION}/experiments/{EXPERIMENT_NAME}/metrics?project={PROJECT}"
)

Vertex AI Experiments:
https://console.cloud.google.com/vertex-ai/locationsus-central1/experiments/chicago-taxi-tips-classifier-v09/metrics?project=supply-chain-twin-349311


## 7. Submit a Hyperparameter Tuning Job to Vertex AI

For more information about configuring a hyperparameter study, refer to [Vertex AI Hyperparameter job configuration](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning).

### Configure a hyperparameter job

In [70]:
metric_spec = {
    'ACCURACY': 'maximize'
}

parameter_spec = {
    'learning-rate': hp_tuning.DoubleParameterSpec(min=0.0001, max=0.01, scale='log'),
    'hidden-units': hp_tuning.CategoricalParameterSpec(values=["32,32", "64,64", "128,128"])
}

In [71]:
tuning_job_display_name = f"hpt_{TRAINER_PACKAGE_NAME}_{run_id}"

hp_tuning_job = vertex_ai.HyperparameterTuningJob(
    display_name=tuning_job_display_name,
    custom_job=training_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=4,
    parallel_trial_count=2,
    search_algorithm=None # Bayesian optimization.
)

### Submit the hyperparameter tuning job

In [ ]:
print("Submitting a hyperparameter tunning job...")

hp_tuning_job.run(
    service_account=SERVICE_ACCOUNT,
    tensorboard=tensorboard_resource_name,
    restart_job_on_worker_restart=False,
    sync=True,
)

Submitting a hyperparameter tunning job...
Creating HyperparameterTuningJob


INFO:google.cloud.aiplatform.jobs:Creating HyperparameterTuningJob


HyperparameterTuningJob created. Resource name: projects/1049330678395/locations/us-central1/hyperparameterTuningJobs/7486297356078415872


INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob created. Resource name: projects/1049330678395/locations/us-central1/hyperparameterTuningJobs/7486297356078415872


To use this HyperparameterTuningJob in another session:


...

INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob projects/1049330678395/locations/us-central1/hyperparameterTuningJobs/7486297356078415872 current state:
JobState.JOB_STATE_RUNNING


HyperparameterTuningJob projects/1049330678395/locations/us-central1/hyperparameterTuningJobs/7486297356078415872 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob projects/1049330678395/locations/us-central1/hyperparameterTuningJobs/7486297356078415872 current state:
JobState.JOB_STATE_RUNNING


HyperparameterTuningJob projects/1049330678395/locations/us-central1/hyperparameterTuningJobs/7486297356078415872 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob projects/1049330678395/locations/us-central1/hyperparameterTuningJobs/7486297356078415872 current state:
JobState.JOB_STATE_RUNNING


### Retrieve trial results

In [73]:
hp_tuning_job.trials

[id: "1"
 state: SUCCEEDED
 parameters {
   parameter_id: "hidden-units"
   value {
     string_value: "64,64"
   }
 }
 parameters {
   parameter_id: "learning-rate"
   value {
     number_value: 0.0010000000000000002
   }
 }
 final_measurement {
   step_count: 5120
   metrics {
     metric_id: "ACCURACY"
     value: 0.8720609545707703
   }
 }
 start_time {
   seconds: 1719306112
   nanos: 606885843
 }
 end_time {
   seconds: 1719310661
 },
 id: "2"
 state: SUCCEEDED
 parameters {
   parameter_id: "hidden-units"
   value {
     string_value: "32,32"
   }
 }
 parameters {
   parameter_id: "learning-rate"
   value {
     number_value: 0.0003631131548707128
   }
 }
 final_measurement {
   step_count: 5120
   metrics {
     metric_id: "ACCURACY"
     value: 0.8548821806907654
   }
 }
 start_time {
   seconds: 1719306112
   nanos: 607135495
 }
 end_time {
   seconds: 1719311146
 },
 id: "3"
 state: SUCCEEDED
 parameters {
   parameter_id: "hidden-units"
   value {
     string_value: "32,32"

In [74]:
best_trial = sorted(
    hp_tuning_job.trials, 
    key=lambda trial: trial.final_measurement.metrics[0].value, 
    reverse=True
)[0]

print("Best trial ID:", best_trial.id)
print("Validation Accuracy:", best_trial.final_measurement.metrics[0].value)
print("Hyperparameter Values:")
for parameter in best_trial.parameters:
    print(f" - {parameter.parameter_id}:{parameter.value}")

Best trial ID: 1
Validation Accuracy: 0.8720609545707703
Hyperparameter Values:
 - hidden-units:64,64
 - learning-rate:0.0010000000000000002
